In [2]:
import pandas as pd 
import numpy as np
import os

In [20]:
from pymongo import MongoClient
def get_database(file_path):
 
   # Open the file in read mode
   with open(file_path, 'r') as file:
      # Read the entire content of the file
      file_content = file.read()

   
      # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
      client = MongoClient(file_content)
   
   # Create the database for our example (we will use the same database throughout the tutorial
   return client['rain_gauges']
  
# This is added so that many files can reuse the function get_database()
if __name__ == "__main__":   
  
   # Get the database
   dbname = get_database()

In [22]:
dbname

Database(MongoClient(host=['ac-halpwhw-shard-00-02.er4docs.mongodb.net:27017', 'ac-halpwhw-shard-00-00.er4docs.mongodb.net:27017', 'ac-halpwhw-shard-00-01.er4docs.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-fp6tp5-shard-0', tls=True), 'rain_gauges')

In [64]:
collection = dbname.get_collection('inmet_estacoes')

In [51]:
PATH_TO_FILE = '/home/ayumi/final_project_university/analytics-data/rain_gauges'

In [62]:
len(file_name_list)

567

In [78]:

file_name_list = os.listdir(PATH_TO_FILE)
end_value = min(567, len(file_name_list))
# Iterar sobre os intervalos de 10 em 10 até o valor 567 ou o número total de arquivos
for start_index in range(0, end_value, 10):
    end_index = min(start_index + 9, end_value - 1)
    lista = []
    
    for file_name in file_name_list[start_index:end_index + 1]:
        # Seu código atual aqui
        metadata = pd.read_csv(os.path.join(PATH_TO_FILE, file_name),
                               header=0,
                               nrows=8,
                               encoding='iso-8859-1',
                               sep=';',
                               decimal=',',
                               on_bad_lines='skip')

        df = pd.read_csv(os.path.join(PATH_TO_FILE, file_name),
                         header=8,
                         encoding='iso-8859-1',
                         sep=';',
                         decimal=',')

        rain_by_month = df[['Data', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']].groupby('Data')[
            'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].sum()

        lista.append({
            'nome_estacao': file_name[:-4],
            'UF': metadata.iloc[0, 1],
            'ESTACAO': metadata.iloc[1, 1],
            'CODIGO': metadata.iloc[2, 1],
            'LATITUDE': float(metadata.iloc[3, 1].replace(',', '.')),
            'LONGITUDE': float(metadata.iloc[4, 1].replace(',', '.')),
            'PLUVIO_ORIGEM': metadata.iloc[6, 1],
            'acumulado_dia': dict(zip(rain_by_month.index, rain_by_month))
        })
    collection.insert_many(lista)
        